In [2]:
from __init__ import PRP; import sys
sys.path.append(PRP)
sys.path.append(PRP + 'scripts/')

from scripts.grad_compare import *


# from veros import runtime_settings
# setattr(runtime_settings, 'backend', 'jax')
# setattr(runtime_settings, 'force_overwrite', True)
# setattr(runtime_settings, 'linear_solver', 'scipy_jax')

from setups.acc.acc_learning import ACCSetup
from tqdm import tqdm



Importing core modules
 Using computational backend jax on cpu
  Kernels are compiled during first iteration, be patient
 Runtime settings are now locked



In [4]:
acc = ACCSetup()
acc.setup()

for step in tqdm(range(300)) :
    acc.step(acc.state)

Running model setup
Diffusion grid factor delta_iso1 = 0.01942284820457075


100%|██████████| 300/300 [00:09<00:00, 32.52it/s]


In [ ]:
initial_state = acc.state.copy()

target_state = initial_state.copy()

for step in tqdm(range(3)) :
    acc.step(target_state)

target_state.variables.temp[:,:,:,1].mean()

100%|██████████| 3/3 [00:00<00:00, 11.00it/s]


Array(7.45361847, dtype=float64)

In [42]:
perturb_state = initial_state.copy()

with perturb_state.variables.unlock() :
    perturb_state.variables.temp *= 1.3

for step in tqdm(range(3)) :
    acc.step(perturb_state)

final_perturb_state = perturb_state.copy()

final_perturb_state.variables.temp[:,:,:,1].mean()

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 13.16it/s]


Array(9.68711851, dtype=float64)

# Gradients 

In [ ]:
# Params auto-diff
def agg_sum(state, target_state, key_sum = 'temp') :
    return ((getattr(state.variables, key_sum)[...,1] - getattr(target_state.variables, key_sum)[...,1])**2).mean()

step_function = acc.step
agg_function = lambda state : agg_sum(state, target_state, key_sum='temp')


vjpm = vjp_grad(step_function, agg_function, var_name)

In [75]:
perturb_state = initial_state.copy()
target_field = initial_state.variables.temp.copy()

with perturb_state.variables.unlock() :
    perturb_state.variables.temp *= 1.3

field = perturb_state.variables.temp.copy()

In [79]:
stats = []
for i in range(200) : 
    output_forward, gradients = vjpm.g(perturb_state, var_value = field, iterations=2, var_name = 'temp')
    distance = ((field - target_field)**2).mean()
    print(f'loss : {output_forward} distance eval : {distance}')
    stats.append({'loss' : output_forward, 'distance' : distance})
    field -= 1.0 * gradients

loss : 7.69123618186966 distance eval : 7.716226089617581
loss : 7.691079698513828 distance eval : 7.7161823772044436
loss : 7.690923529836893 distance eval : 7.716138665680953
loss : 7.690767290058912 distance eval : 7.716094955045168
loss : 7.690611026493657 distance eval : 7.716051245297334
loss : 7.690454781319037 distance eval : 7.716007536437663
loss : 7.69029870214477 distance eval : 7.7159638284796195
loss : 7.6901425035697395 distance eval : 7.715920121394601
loss : 7.689986301946964 distance eval : 7.715876415196926
loss : 7.689830111650173 distance eval : 7.7158327098872475
loss : 7.689673924549991 distance eval : 7.715789005464916
loss : 7.689517742480433 distance eval : 7.715745301930386
loss : 7.689361583451092 distance eval : 7.7157015992840865
loss : 7.689205410870526 distance eval : 7.715657897522437
loss : 7.6890492441409855 distance eval : 7.715614196650683
loss : 7.688893086460918 distance eval : 7.715570496665876
loss : 7.688736907274526 distance eval : 7.715526797

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x107b6fcb0>>
Traceback (most recent call last):
  File "/Users/emeunier/miniforge3/envs/veros/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


loss : 7.68811228674884 distance eval : 7.7153520101278685
loss : 7.687956158265357 distance eval : 7.715308315468617
loss : 7.687800046950786 distance eval : 7.7152646216952965
loss : 7.68764394169899 distance eval : 7.715220928815158
loss : 7.6874878282613475 distance eval : 7.715177236817392
loss : 7.687331716342502 distance eval : 7.715133545705765
loss : 7.687175638767339 distance eval : 7.715089855482272
loss : 7.687019547144327 distance eval : 7.71504616616578
loss : 7.68686346115875 distance eval : 7.715002477716581
loss : 7.68670737484111 distance eval : 7.714958790153419
loss : 7.6865513099653064 distance eval : 7.7149151034762395
loss : 7.686395214689149 distance eval : 7.714871417687474
loss : 7.68623914564372 distance eval : 7.714827732793231
loss : 7.686083076107605 distance eval : 7.714784048777927
loss : 7.685927062511412 distance eval : 7.714740365650737
loss : 7.685771062908711 distance eval : 7.7146966834218045
loss : 7.685615009620913 distance eval : 7.7146530020674

: 

In [ ]:
stats

In [72]:
print(output_forward)

7.36808190758858e-31


In [63]:
output_forward

Array(85.24857613, dtype=float64)